In [1]:
import logging
import os
import math

import numpy as np

import torch
from torch.utils.data import Dataset


In [2]:
logger = logging.getLogger(__name__)

In [5]:
def seq_collate(data):
    (obs_seq_list, pred_seq_list, obs_seq_rel_list, pred_seq_rel_list,
     non_linear_ped_list, loss_mask_list) = zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end]
                     for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    # Data format: batch, input_size, seq_len
    # LSTM input format: seq_len, batch, input_size
    obs_traj = torch.cat(obs_seq_list, dim=0).permute(2, 0, 1) # permute : 인덱스들을 바꾸고자 하는 위치로 변경
    pred_traj = torch.cat(pred_seq_list, dim=0).permute(2, 0, 1)
    obs_traj_rel = torch.cat(obs_seq_rel_list, dim=0).permute(2, 0, 1)
    pred_traj_rel = torch.cat(pred_seq_rel_list, dim=0).permute(2, 0, 1)
    non_linear_ped = torch.cat(non_linear_ped_list)
    loss_mask = torch.cat(loss_mask_list, dim=0)
    seq_start_end = torch.LongTensor(seq_start_end)
    
    out = [
        obs_traj, pred_traj, obs_traj_rel, pred_traj_rel, non_linear_ped,
        loss_mask, seq_start_end
    ]

    return tuple(out)

In [6]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [7]:
def poly_fit(traj, traj_len, 
             threshold
             ):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len) # np.linspace(시작점, 끝점, 구간 내 숫자 개수)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold: # error
       return 1.0
    else:
       return 0.0

In [8]:
class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""
    def __init__(
        self, data_dir, obs_len=8, pred_len=12, skip=1, 
        threshold=0.002, min_ped=1, 
        delim='\t'
    ):
        """
        Args:
        - data_dir: Directory containing dataset files in the format
        <frame_id> <agent_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non linear traj
        when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a seqeunce
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        # self.data_dir = data_dir
        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim

        all_files = os.listdir(self.data_dir)
        # print(all_files) : data_dir에 있는 파일 목록 출력
        all_files = [os.path.join(self.data_dir, _path) for _path in all_files]
        
        num_peds_in_seq = []
        seq_list = []
        seq_list_rel = []
        loss_mask_list = []
        non_linear_ped = []
        
        for path in all_files:
            data = read_file(path, delim)
            frames = np.unique(data[:, 0]).tolist() # frame 번호
            frame_data = []
            for frame in frames:
                frame_data.append(data[frame == data[:, 0], :]) # frame_data, 각 frame별 frame_data (agent 정보, x, y) --> 2차원 list
            num_sequences = int(
                math.ceil((len(frames) - self.seq_len + 1) / skip))

            for idx in range(0, num_sequences * self.skip + 1, skip):
                curr_seq_data = np.concatenate(
                    frame_data[idx:idx + self.seq_len], axis=0) # seq_len에 해당하는 frame_data를 합침
                peds_in_curr_seq = np.unique(curr_seq_data[:, 1]) # 현재 seq에 있는 agent 목록
                curr_seq_rel = np.zeros((len(peds_in_curr_seq), 2, # (현재 seq에 있는 agent 수, 2, seq_len) -> seq에 있는 agent들의 seq_len 동안의 좌표
                                         self.seq_len))
                curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len)) # (현재 seq에 있는 agent 수, 2, seq_len)
                curr_loss_mask = np.zeros((len(peds_in_curr_seq), # (현재 seq에 있는 agent 수, seq_len)
                                           self.seq_len))
                num_peds_considered = 0
                _non_linear_ped = []
                for _, ped_id in enumerate(peds_in_curr_seq):
                    curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] ==
                                                 ped_id, :]
                    curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                    pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                    pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                    if pad_end - pad_front != self.seq_len:
                        continue
                    curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
                    curr_ped_seq = curr_ped_seq
                    # Make coordinates relative
                    rel_curr_ped_seq = np.zeros(curr_ped_seq.shape)
                    rel_curr_ped_seq[:, 1:] = \
                        curr_ped_seq[:, 1:] - curr_ped_seq[:, :-1]
                    _idx = num_peds_considered
                    curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
                    curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_ped_seq
                    # Linear vs Non-Linear Trajectory
                    _non_linear_ped.append(
                        poly_fit(curr_ped_seq, pred_len, threshold))
                    curr_loss_mask[_idx, pad_front:pad_end] = 1
                    num_peds_considered += 1

                if num_peds_considered > min_ped:
                    non_linear_ped += _non_linear_ped
                    num_peds_in_seq.append(num_peds_considered)
                    loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                    seq_list.append(curr_seq[:num_peds_considered])
                    seq_list_rel.append(curr_seq_rel[:num_peds_considered])

        self.num_seq = len(seq_list)
        seq_list = np.concatenate(seq_list, axis=0)
        seq_list_rel = np.concatenate(seq_list_rel, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)

        # Convert numpy -> Torch Tensor
        self.obs_traj = torch.from_numpy(
            seq_list[:, :, :self.obs_len]).type(torch.float)
        self.pred_traj = torch.from_numpy(
            seq_list[:, :, self.obs_len:]).type(torch.float)
        self.obs_traj_rel = torch.from_numpy(
            seq_list_rel[:, :, :self.obs_len]).type(torch.float)
        self.pred_traj_rel = torch.from_numpy(
            seq_list_rel[:, :, self.obs_len:]).type(torch.float)
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [
            (start, end)
            for start, end in zip(cum_start_idx, cum_start_idx[1:])
        ]
    def __len__(self):
        return self.num_seq
    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        out = [
            self.obs_traj[start:end, :], self.pred_traj[start:end, :],
            self.obs_traj_rel[start:end, :], self.pred_traj_rel[start:end, :],
            self.non_linear_ped[start:end], self.loss_mask[start:end, :]
        ]
        return out

In [9]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 8
        self.skip = 1
        # Optimization
        self.batch_size = 4
        self.num_iterations = 1000 # 10000
        self.num_epochs = 50 # 200                     
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        # self.default_backbone= 'resnet18'
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "1" # type=str   


args = CreateArg() 

In [19]:
import sys
# sys.path.append('/home/gpuadmin/dev/traj_pred/Trajectory_Prediction/traffino')
sys.path.append('C:/Users/NGN/dev/Traffino/TRAFFINO/')

In [51]:
from data.loader_basic import data_loader # basic train을 위한 loader
temp_path = 'C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/waterloo/temp'
logger.info("Initializing temp dataset")
train_dset, train_loader = data_loader(args, temp_path) # train_dset은 TrajectoryDataset, train_loader는 DataLoader


['temp.txt']
C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/waterloo/temp\temp.txt


In [52]:
print(len(train_dset))
print(len(train_loader))

195
49


In [53]:
print(type(train_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [54]:
i=0
for batch in train_loader: # batch
    print(i)

    batch = [tensor.cuda() for tensor in batch]
    (obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,
     loss_mask, seq_start_end) = batch
    i+=1
    if(i==1):
        print(f'Read {i}-th data done')
        break
            
        

0
Read 1-th data done


In [58]:
# obs_traj

print(obs_traj)
print(f"len(obs_traj): {len(obs_traj)}")
print(obs_traj.ndim)
print(f"obs_traj.shape: {obs_traj.shape}")
print(type(obs_traj))

tensor([[[10.3100,  5.9700],
         [12.4900,  6.6000],
         [ 9.5700,  6.2400],
         [11.9400,  6.7700],
         [ 8.7300,  6.3400],
         [11.0300,  6.8400],
         [11.8400,  5.8200],
         [ 7.9400,  6.5000],
         [10.2100,  6.8100],
         [10.9100,  5.9200]],

        [[ 9.5700,  6.2400],
         [11.9400,  6.7700],
         [ 8.7300,  6.3400],
         [11.0300,  6.8400],
         [ 7.9400,  6.5000],
         [10.2100,  6.8100],
         [10.9100,  5.9200],
         [ 7.1700,  6.6200],
         [ 9.3600,  6.8500],
         [10.0000,  5.8900]],

        [[ 8.7300,  6.3400],
         [11.0300,  6.8400],
         [ 7.9400,  6.5000],
         [10.2100,  6.8100],
         [ 7.1700,  6.6200],
         [ 9.3600,  6.8500],
         [10.0000,  5.8900],
         [ 6.4700,  6.6800],
         [ 8.5900,  6.8500],
         [ 9.0900,  6.0200]],

        [[ 7.9400,  6.5000],
         [10.2100,  6.8100],
         [ 7.1700,  6.6200],
         [ 9.3600,  6.8500],
        

In [56]:
j=0
for batch in train_dset: # batch
    print(j)

    batch = [tensor.cuda() for tensor in batch]
    (obs_traj_td, pred_traj_gt_td, obs_traj_rel_td, pred_traj_gt_rel_td, non_linear_ped_Td,
     loss_mask_td) = batch
    j+=1
    if(j==1):
        print(f'Read {j}-th data done')
        break



0
Read 1-th data done


In [59]:
# obs_traj

print(obs_traj_td)
print(f"len(obs_traj_td): {len(obs_traj_td)}")
print(obs_traj_td.ndim)
print(f"obs_traj_td.shape: {obs_traj_td.shape}")
print(type(obs_traj_td))

tensor([[[10.3100,  9.5700,  8.7300,  7.9400,  7.1700,  6.4700,  5.8600,
           5.2400],
         [ 5.9700,  6.2400,  6.3400,  6.5000,  6.6200,  6.6800,  6.8200,
           6.9800]],

        [[12.4900, 11.9400, 11.0300, 10.2100,  9.3600,  8.5900,  7.7800,
           6.9600],
         [ 6.6000,  6.7700,  6.8400,  6.8100,  6.8500,  6.8500,  6.8400,
           6.8400]]], device='cuda:0')
len(obs_traj_td): 2
3
obs_traj_td.shape: torch.Size([2, 2, 8])
<class 'torch.Tensor'>
